<a href="https://colab.research.google.com/github/johanhoffman/DD2363-VT20/blob/HelmerNylen/Lab-3/HelmerNylen_lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 3: Iterative Methods**
**Helmer Nylén**

# **Abstract**

We implement four iterative methods and verify their correctness: Jacobi iteration, Gauss-Seidel iteration, Newton's method, and the GMRES method. All of these were found to be correct, but Newton's method does have some problems with numerical accuracy in the tests used.

#**About the code**

A short statement on who is the author of the file, and if the code is distributed under a certain license. 

In [99]:
"""This program is a template for lab reports in the course"""
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2019 Helmer Nylén (helmern@kth.se)

# This file is part of the course DD2363 Methods in Scientific Computing
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

To have access to the neccessary modules you have to run this cell. If you need additional modules, this is where you add them. 

In [0]:
# Load neccessary modules.
import numpy as np
from random import randint
from numpy.polynomial import polynomial as P
from functools import reduce

# **Introduction**

In this report we implement four iterative methods: Jacobi iteration, Gauss-Seidel iteration, a scalar version of Newton's method, and the GMRES method. These are based on algorithms described in the lecture notes combined with `numpy` objects and functionality.


# **Methods**

Chapter 7.3 describes fixed-point iteration using an algorithm known as _Richardson iteration_. We can use this as a basis for both Jacobi iteration and Gauss-Seidel iteration by preconditioning the iteration matrix with an approximate inverse. These methods are outlined below.  

In [0]:
# Algorithm 7.1, chapter 7.3
def _richardson_iteration(A, b, alpha, tolerance = 1e-10):
  assert A.shape[0] == b.shape[0] and 1 > alpha > 0 and tolerance > 0

  x = np.zeros(A.shape[1])
  r = None
  while r is None or np.linalg.norm(r) > tolerance:
    r = b - A @ x
    x += alpha * r
  return x

def _left_precond(A, B, b, alpha, tolerance = 1e-10):
  return _richardson_iteration(B @ A, B @ b, alpha, tolerance)

def _right_precond(A, B, b, alpha, tolerance = 1e-10):
  x = _richardson_iteration(A @ B, b, alpha, tolerance)
  return B @ x

According to the lecture notes page 133, Jacobi iteration is equivalent to left preconditioned Richardson iteration using $B = (\alpha D)^{-1}$, where $D$ is a diagonal matrix with the same elements as the diagonal of $A$. Given our above functions this yields a simple implementation of Jacobi iteration:

In [0]:
def jacobi(A, b, alpha = 0.05, tolerance = 1e-10, richardson = True):
  Dvec = np.diag(A)
  assert not (Dvec == 0).any()
  Dvec = Dvec * alpha
  Dinv = np.diag(1 / Dvec)
  return _left_precond(A, Dinv, b, alpha, tolerance)

We can only be certain that the Jacobi iteration will converge when the spectral radius $\rho$ of the iteration matrix $M_j$ is less than one, i.e. $$\rho(M_j) = \rho(I - D^{-1} A) < 1.$$ We ensure that this is the case before testing our random matrix $A$.

We also know that "convergence of the Jacobi iteration depends on how close $A$ is to a diagonal matrix" (lecture notes page 133), and as such make sure to emphasize the diagonal when generating $A$.

We verify convergence by asserting that both the residual and that the distance to the manufactured solution is small.

In [0]:
def _spectral_radius(A):
  return max(map(abs, np.linalg.eigvals(A)))

def test_jacobi(richardson = True):
  n = randint(1, 10)
  A = None
  while A is None or _spectral_radius(np.eye(n) - np.diag(1 / np.diag(A)) @ A) >= 1:
    A = np.random.rand(n, n)
    Adiag = 10 * np.random.rand(n) + 1
    A = A + np.diag(Adiag)
    # make A symmetric
    for i in range(n):
      for j in range(i):
        A[i, j] = A[j, i]

  y = np.random.rand(n)
  b = A @ y
  x = jacobi(A, b, richardson = richardson)

  return np.isclose(0, np.linalg.norm(A @ x - b), atol=1e-10) and \
         np.isclose(0, np.linalg.norm(x - y), atol=1e-4)

Similarly to Jacobi iteration, Gauss-Seidel iteration can be defined via Richardson iteration left-preconditioned with $B = (\alpha L)^{-1}$ where $L$ is the lower triangular matrix with its nonzero elements $l_{ij} = a_{ij}$, $i \geq j$. Inverting $L$ can be done via forward substitution as defined below.

In [0]:
def _forward_substitution(L, b):
  n = L.shape[1]
  x = np.zeros(n)
  for i in range(n):
    s = x.dot(L[i, :])
    x[i] = (b[i] - s) / L[i, i]
  return x

def _inv(L):
  n = L.shape[0]
  eye = np.eye(n)
  inv = np.zeros(L.shape)
  for i in range(n):
    inv[:, i] = _forward_substitution(L, eye[:, i])
  return inv

The definition of Gauss-Seidel iteration is then trivial given the already defined functions.

In [0]:
def gauss_seidel(A, b, alpha = 0.05):
  L = np.tril(A)
  Linv = _inv(alpha * L)
  return _left_precond(A, Linv, b, alpha)

Again, we can only be certain of convergence when $$\rho(M_{GS}) = \rho(I-L^{-1}A) < 1,$$ and thus use matrices $A$ which satisfy that condition. We generate these matrices in a similar manner to the Jacobi tests and verify the same quantities for convergence.

In [0]:
def test_gs():
  n = randint(1, 10)
  A = None
  while A is None or _spectral_radius(np.eye(n) - np.linalg.inv(np.tril(A)) @ A) >= 1:
    A = np.random.rand(n, n)
    Adiag = 10 * np.random.rand(n) + 1
    A = A + np.diag(Adiag)
    # make A symmetric
    for i in range(n):
      for j in range(i):
        A[i, j] = A[j, i]

  y = np.random.rand(n)
  b = A @ y
  x = gauss_seidel(A, b)
  
  return np.isclose(0, np.linalg.norm(A @ x - b), atol=1e-10) and \
         np.isclose(0, np.linalg.norm(x - y), atol=1e-4)

For Newton's method, we begin by defining a numerical derivative using the central difference.

In [0]:
def _derivative(f, x, h = 1e-12):
  return (f(x + h) - f(x - h)) / (2 * h)

We then use this in our implementation of algorithm 8.2, chapter 8.3 of the lecture notes.

In [0]:
# Algorithm 8.2, chapter 8.3
def newton_scalar(f, x0 = 0, tolerance = 1e-10):
  assert hasattr(f, '__call__')
  x = x0
  while abs(f(x)) > tolerance:
    df = _derivative(f, x)
    x -= f(x) / df
  return x

To test this method we create a polynomial $f$ of a certain degree $n$ with known zeroes $\{z_i\}$. Given an arbitrary $x_0$ in the domain of $f$ we can then ensure that the $x$ the algorithm yields satisfies $$f(x) \approx 0$$ and $$|x-z_i| \approx 0$$ for some $z_i$.

In [0]:
def test_newton():
  n = randint(1, 5)
  zeros = np.random.rand(n) * 20 - 10
  polynomials = [(-zeros[i], 1) for i in range(n)]
  pol = reduce(P.polymul, polynomials)
  f = lambda x: P.polyval(x, pol)
  try:
    for _ in range(n):
      x = newton_scalar(f, x0 = np.random.rand() * 20 - 10)
      assert np.isclose(0, abs(f(x)), atol=1e-10)               # x is a zero of f
      assert np.isclose(0, min(map(abs, x - zeros)), atol=1e-4) # x is close to a listed zero
    return True
  except AssertionError:
    return False

For the GMRES method we implement algorithms 7.2 and 7.3, chapter 7.4, for GMRES and Arnoldi iteration, respectively. The specifics of the algorithms are modified slightly due to minor errors in the lecture notes.

In [0]:
def _standard_basis(k, i=0):
  return (np.array(range(k)) == i) * 1

def gmres(A, b, tolerance = 1e-10):
  k = 1
  r = None
  while r is None or np.linalg.norm(r) / np.linalg.norm(b) > tolerance:
    Q, H = arnoldi(A, b, k)
    y = np.linalg.lstsq(H, np.linalg.norm(b) * _standard_basis(k + 1))[0]
    r = H @ y
    r = np.linalg.norm(b) * _standard_basis(k + 1) - r
    k += 1
  return Q[:, 0:k-1] @ y

def arnoldi(A, b, k):
  Q = np.zeros((b.shape[0], k+1))
  H = np.zeros((k + 1, k))
  Q[:, 0] = b / np.linalg.norm(b)
  for j in range(k):
    v = A @ Q[:, j]
    for i in range(j + 1):
      H[i, j] = Q[:, i].dot(v)
      v = v - H[i, j] * Q[:, i]
    H[j + 1, j] = np.linalg.norm(v)
    Q[:, j + 1] = v / H[j + 1, j]
  return Q, H

The GMRES method is not as dependent on the properties of $A$ as the Jacobi or Gauss-Seidel iterations, and thus we need not ensure anything in particular about our randomized $A$ before testing it.

In [0]:
def test_gmres():
  n = randint(1, 10)
  A = np.random.rand(n, n)

  y = np.random.rand(n)
  b = A @ y
  x = gmres(A, b)
  
  return np.isclose(0, np.linalg.norm(A @ x - b), atol=1e-10) and \
         np.isclose(0, np.linalg.norm(x - y), atol=1e-4)

# **Results**

In [112]:
total_tests = 1000
jacobi_sum = 0
gs_sum = 0
newton_sum = 0
gmres_sum = 0
for i in range(total_tests):
  #print(str(i).rjust(3), end=" \n"[(i+1) % 100 == 0])
  jacobi_sum += test_jacobi()
  gs_sum += test_gs()
  newton_sum += test_newton()
  gmres_sum += test_gmres()

print(f"{jacobi_sum} of {total_tests} jacobi tests passed")
print(f"{gs_sum} of {total_tests} gauss-seidel tests passed")
print(f"{newton_sum} of {total_tests} newton tests passed")
print(f"{gmres_sum} of {total_tests} gmres tests passed")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in double_scalars
  
/usr/local/lib/python3.6/dist-packages/numpy/polynomial/polynomial.py:733: RuntimeWarning: invalid value encountered in double_scalars
  c0 = c[-1] + x*0


1000 of 1000 jacobi tests passed
1000 of 1000 gauss-seidel tests passed
991 of 1000 newton tests passed
1000 of 1000 gmres tests passed


We note that all Jacobi, Gauss-Seidel and GMRES tests tend to pass while a few of the Newton tests fail. We also note a number of error messages from numpy.

# **Discussion**

As all tests for $Ax=b$ solvers pass we assume these to be correct.

The motivation for using polynomials to test the implementation of Newton's method was that all analytic scalar functions can be arbitrarily well approximated with a polynomial using Taylor expansion (Taylor's theorem). However, during testing with zeros of multiplicity greater than one the method failed much more often. This is likely due to $$f'(x) = f(x) = 0$$ at zeroes of $f$, which hinders convergence. The fact that the method still fails occasionally despite that all zeroes have a multiplicity of one can likely be attributed to numerical errors, as we are dealing with very small numbers (close to the machine epsilon listed below).

In [113]:
import sys; sys.float_info.epsilon

2.220446049250313e-16